In [ ]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:10 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages
Get:12 https://developer.downl

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Automotive_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show(truncate=False)

+-----------+-----------+--------------+----------+--------------+---------------------------------------------------------------------------------------------------------------------+----------------+-----------+-------------+-----------+----+-----------------+---------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
# Recreate the vine_table
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show(truncate=False)

+--------------+-----------+-------------+-----------+----+-----------------+
|review_id     |star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|RAB23OVFNCXZQ |1          |0            |0          |N   |Y                |
|R3NORADVJO6IE6|5          |0            |0          |N   |Y                |
|R299F4SO98S5OO|5          |1            |1          |N   |Y                |
|R2DA9DOT03UW6I|5          |2            |3          |N   |Y                |
|R2OGCH681EQHU6|5          |0            |0          |N   |Y                |
|R2JMKIC16MHD7 |5          |2            |2          |N   |Y                |
|R1DB5DA7CWWTI8|5          |0            |0          |N   |Y                |
|R1N8XWFDK4QACP|5          |0            |0          |N   |Y                |
|R19VFLX6MRU0CX|5          |0            |0          |N   |Y                |
|RLTD8YDGTP9PT |1          |0            |0          |N   |Y    

In [ ]:
# Filter the DataFrame and create a new DataFrame where the total_votes count is equal to or greater than 20
from pyspark.sql.functions import col
filtered_df = vine_df.filter(col("total_votes") >= "20")
filtered_df.show(truncate=False)

+--------------+-----------+-------------+-----------+----+-----------------+
|review_id     |star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1T4FVSABO0IDP|5          |8            |31         |N   |Y                |
|R397VXR1GAK6C9|5          |21           |21         |N   |Y                |
|R2H6KB9RHS17GA|3          |59           |69         |N   |Y                |
|R3HAK0MUN9F5IT|1          |6            |28         |N   |Y                |
|R29UTZTOPUVRSV|5          |21           |21         |N   |Y                |
|RY9FLUD8VG6XD |4          |27           |32         |N   |Y                |
|R3Q5NAY8BGAKHI|5          |34           |40         |N   |Y                |
|RRVJ7TYDEK7EO |5          |30           |30         |N   |Y                |
|R2S4CHISQA2WDH|5          |17           |20         |N   |Y                |
|R1JNALJ77OULPH|4          |57           |65         |N   |Y    

In [ ]:
# Filter the DataFrame created in step 1 and create a new DataFrame where the number of helpful_votes divided by total_votes is equal to or greater than 50%
percentage_votes_df = filtered_df.filter((col("helpful_votes")/col("total_votes")) >= "0.5")
percentage_votes_df.show(truncate=False)

+--------------+-----------+-------------+-----------+----+-----------------+
|review_id     |star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R397VXR1GAK6C9|5          |21           |21         |N   |Y                |
|R2H6KB9RHS17GA|3          |59           |69         |N   |Y                |
|R29UTZTOPUVRSV|5          |21           |21         |N   |Y                |
|RY9FLUD8VG6XD |4          |27           |32         |N   |Y                |
|R3Q5NAY8BGAKHI|5          |34           |40         |N   |Y                |
|RRVJ7TYDEK7EO |5          |30           |30         |N   |Y                |
|R2S4CHISQA2WDH|5          |17           |20         |N   |Y                |
|R1JNALJ77OULPH|4          |57           |65         |N   |Y                |
|R13YNG792BCR77|5          |19           |20         |N   |Y                |
|R1RYG8MVOPK4Q2|5          |18           |20         |N   |Y    

In [ ]:
# Filter the DataFrame created in step 2 to obtain the rows where a review was written as part of the Vine program (paid), vine == 'Y'.
paid_df = percentage_votes_df.filter(col("vine") == "Y")
paid_df.show(truncate=False)

+--------------+-----------+-------------+-----------+----+-----------------+
|review_id     |star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|REKN4PLM5174P |4          |219          |225        |Y   |N                |
|RQOG8IIJX7CKB |4          |21           |27         |Y   |N                |
|R33EJ9PSZR13CM|4          |26           |30         |Y   |N                |
|RD2BFV896F321 |5          |16           |20         |Y   |N                |
|R37GSXLA11XF8N|5          |30           |34         |Y   |N                |
|RQBQYBM9VMSH4 |5          |19           |20         |Y   |N                |
|R2YE4F9ESKFWEK|3          |20           |23         |Y   |N                |
|R1E4L7HHB8TBP9|4          |80           |92         |Y   |N                |
|R2C2WXDXMZL005|2          |20           |25         |Y   |N                |
|R1PPY4U0K03KTE|5          |31           |38         |Y   |N    

In [ ]:
# Filter the DataFrame created in step 2 to obtain the rows where a review was written as part of the Vine program (unpaid), vine == 'N'.
unpaid_df = percentage_votes_df.filter(col("vine") == "N")
unpaid_df.show(truncate=False)

+--------------+-----------+-------------+-----------+----+-----------------+
|review_id     |star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R397VXR1GAK6C9|5          |21           |21         |N   |Y                |
|R2H6KB9RHS17GA|3          |59           |69         |N   |Y                |
|R29UTZTOPUVRSV|5          |21           |21         |N   |Y                |
|RY9FLUD8VG6XD |4          |27           |32         |N   |Y                |
|R3Q5NAY8BGAKHI|5          |34           |40         |N   |Y                |
|RRVJ7TYDEK7EO |5          |30           |30         |N   |Y                |
|R2S4CHISQA2WDH|5          |17           |20         |N   |Y                |
|R1JNALJ77OULPH|4          |57           |65         |N   |Y                |
|R13YNG792BCR77|5          |19           |20         |N   |Y                |
|R1RYG8MVOPK4Q2|5          |18           |20         |N   |Y    

In [ ]:
# Find the total number of reviews
percentage_votes_df.count()

24824

In [ ]:
# Find the number of 5-star reviews
percentage_votes_df.filter(col("star_rating") == "5").count()

12840

In [13]:
# Find the number of Vine reviews
paid_df.count()

82

In [14]:
# Find the number of non-Vine reviews
unpaid_df.count()

24742

In [15]:
# Find the percentage of 5-star reviews of paid type
(paid_df.filter(col("star_rating") == "5").count())/(paid_df.count())

0.4024390243902439

In [16]:
# Find the percentage of 5-star reviews of unpaid type
(unpaid_df.filter(col("star_rating") == "5").count())/(unpaid_df.count())

0.5176218575701237

In [17]:
# Find the count of vine 5-star reviews
paid_df.filter(col("star_rating") == "5").count()

33

In [18]:
# Find the count of non-vine 5-star reviews
unpaid_df.filter(col("star_rating") == "5").count()

12807